# Data Extraction

In [1]:
import os
from pathlib import Path
from pantograph.server import Server

## Tactic Invocation

Pantograph can extract tactic invocation data from a Lean file. A **tactic
invocation** is a tuple containing the before and after goal states, and the
tactic which converts the "before" state to the "after" state.

To extract tactic invocation data, use `server.tactic_invocations(file_name)`
and supply the file name of the input Lean file.

In [2]:
project_path = Path(os.getcwd()).parent.resolve() / 'examples/Example'
print(f"$PWD: {project_path}")
server = Server(imports=['Example'], project_path=project_path)
units, invocations = server.tactic_invocations(project_path / "Example.lean")

$PWD: /home/aniva/Projects/atp/PyPantograph/examples/Example


The function returns a tuple `([str], [TacticInvocation])`. The former element is a list of strings containing each compilation unit, comment data included.

In [3]:
for i, u in enumerate(units):
    print(f"==== #{i} ====")
    print(u)

==== #0 ====
/-- Ensure that Aesop is running -/
example : α → α :=
  by aesop


==== #1 ====
example : ∀ (p q: Prop), p ∨ q → q ∨ p := by
  intro p q h
  -- Here are some comments
  cases h
  . apply Or.inr
    assumption
  . apply Or.inl
    assumption



The latter is a list of `TacticInvocation`, which contains the `.before` (corresponding to the state before the tactic), `.after` (corresponding to the state after the tactic), and `.tactic` (tactic executed) fields. 

In [4]:
for i in invocations:
    print(f"[Before]\n{i.before}")
    print(f"[Tactic]\n{i.tactic} (using {i.used_constants})")
    print(f"[Afte]\n{i.after}")

[Before]
α : Sort ?u.7
⊢ α → α
[Tactic]
aesop (using [])
[Afte]

[Before]
⊢ ∀ (p q : Prop), p ∨ q → q ∨ p
[Tactic]
intro p q h (using [])
[Afte]
p q : Prop
h : p ∨ q
⊢ q ∨ p
[Before]
p q : Prop
h : p ∨ q
⊢ q ∨ p
[Tactic]
cases h (using ['Eq.refl', 'Or'])
[Afte]
case inl
p q : Prop
h✝ : p
⊢ q ∨ p
case inr p q : Prop h✝ : q ⊢ q ∨ p
[Before]
case inl
p q : Prop
h✝ : p
⊢ q ∨ p
[Tactic]
apply Or.inr (using ['Or.inr'])
[Afte]
case inl.h
p q : Prop
h✝ : p
⊢ p
[Before]
case inl.h
p q : Prop
h✝ : p
⊢ p
[Tactic]
assumption (using [])
[Afte]

[Before]
case inr
p q : Prop
h✝ : q
⊢ q ∨ p
[Tactic]
apply Or.inl (using ['Or.inl'])
[Afte]
case inr.h
p q : Prop
h✝ : q
⊢ q
[Before]
case inr.h
p q : Prop
h✝ : q
⊢ q
[Tactic]
assumption (using [])
[Afte]

